In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold

In [2]:
PROJECT_DIR = r"C:\Data-Science\CampusX\sagemaker-flight-prices-prediction-proj"
DATA_DIR = "data"

In [3]:
os.path.join(PROJECT_DIR, DATA_DIR, "train.csv")

'C:\\Data-Science\\CampusX\\sagemaker-flight-prices-prediction-proj\\data\\train.csv'

In [4]:
def get_data(name):
    file_name = f"{name}.csv"
    file_path = os.path.join(PROJECT_DIR, DATA_DIR, file_name)
    return pd.read_csv(file_path)

In [6]:
flights = get_data("flight_price")
flights.sample(5)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
31,Air India,1/05/2019,Kolkata,Banglore,CCU → GAU → DEL → BLR,09:50,23:15,13h 25m,2 stops,No info,13227
8954,SpiceJet,15/06/2019,Mumbai,Hyderabad,BOM → HYD,05:45,07:15,1h 30m,non-stop,No info,2753
2650,Vistara,9/06/2019,Chennai,Kolkata,MAA → CCU,17:45,20:05,2h 20m,non-stop,No info,11982
4928,IndiGo,3/06/2019,Delhi,Cochin,DEL → HYD → COK,05:05,16:10,11h 5m,1 stop,No info,6818
8584,Jet Airways,15/03/2019,Banglore,New Delhi,BLR → BOM → DEL,08:55,16:10,7h 15m,1 stop,In-flight meal not included,9134


In [7]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


- The dataset contains 10,683 rows and 11 columns.
- Columns 'Route' and 'Total_stops' has missing values
- The datatypes of the columns are object type, which needs to be corrected for datetime columns


## Priliminary analysis

## Exploratory Data Analysis (EDA)


In [8]:
flights.dtypes

Airline            object
Date_of_Journey    object
Source             object
Destination        object
Route              object
Dep_Time           object
Arrival_Time       object
Duration           object
Total_Stops        object
Additional_Info    object
Price               int64
dtype: object

In [9]:
flights.Date_of_Journey.iloc[0]

'24/03/2019'

In [10]:
flights.Dep_Time.iloc[0]

'22:20'

- `Date_of_Journey` needs to be changed to datetime
- `Dep_Time` also needs to be changed to datetime
- `Duration` is a mixed type and `Total_stops` and needs to be changed.


## Check for Duplicates


In [12]:
flights.duplicated().sum()

220

In [19]:
flights[flights.duplicated(keep=False)].sort_values(
    ["Airline", "Date_of_Journey", "Source", "Destination"]
)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6321,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
9848,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
572,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
8168,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
1495,Air India,1/04/2019,Kolkata,Banglore,CCU → DEL → COK → BLR,10:00,01:20 02 Apr,15h 20m,2 stops,No info,10408
...,...,...,...,...,...,...,...,...,...,...,...
2692,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
2870,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
3711,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,20:30,23:20,2h 50m,non-stop,No check-in baggage included,3873
2634,Vistara,24/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,5403


- There are 220 duplicate rows in the dataset.


### Detailed Analysis


### Cleaning Operations


In [116]:
def convert_to_minutes(ser):
    return (
        ser.str.split(" ", expand=True)
        .set_axis(["hour", "minute"], axis=1)
        .assign(
            hour=lambda df_: (df_.hour.str.replace("h", "").astype(int).mul(60)),
            minute=lambda df_: (df_.minute.str.replace("m", "").fillna(0).astype(int)),
        )
        .sum(axis=1)
    )

In [149]:
def clean_data(df):
    return (
        df.drop(index=[6474])
        .drop_duplicates()
        .assign(**{col: df[col].str.strip() for col in df.select_dtypes(include="O")})
        .rename(columns=str.lower)
        .assign(
            airline=lambda df_: (
                df_.airline.str.replace(" Premium economy", "")
                .str.replace(" Business", "")
                .str.title()
            ),
            date_of_journey=lambda df_: pd.to_datetime(
                df_.date_of_journey, dayfirst=True
            ),
            dep_time=lambda df_: pd.to_datetime(df_.dep_time).dt.time,
            arrival_time=lambda df_: pd.to_datetime(df_.arrival_time).dt.time,
            duration=lambda df_: df_.duration.pipe(convert_to_minutes),
            total_stops=lambda df_: (
                df_.total_stops.str.replace("non-stop", "0")
                .str.replace(" stops?", "", regex=True)  # " stop" or " stops"
                .pipe(lambda ser: pd.to_numeric(ser))
            ),
            additional_info=lambda df_: df_.additional_info.str.replace(
                "No info", "No Info"
            ),
        )
        .drop(columns="route")  # as it's not adding any value here..
    )

In [150]:
flights_cleaned = clean_data(flights)  # .additional_info.unique()
flights_cleaned

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-03-24,Banglore,New Delhi,22:20:00,01:10:00,170,0.0,No Info,3897
1,Air India,2019-05-01,Kolkata,Banglore,05:50:00,13:15:00,445,2.0,No Info,7662
2,Jet Airways,2019-06-09,Delhi,Cochin,09:25:00,04:25:00,1140,2.0,No Info,13882
3,Indigo,2019-05-12,Kolkata,Banglore,18:05:00,23:30:00,325,1.0,No Info,6218
4,Indigo,2019-03-01,Banglore,New Delhi,16:50:00,21:35:00,285,1.0,No Info,13302
...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,2019-04-09,Kolkata,Banglore,19:55:00,22:25:00,150,0.0,No Info,4107
10679,Air India,2019-04-27,Kolkata,Banglore,20:45:00,23:20:00,155,0.0,No Info,4145
10680,Jet Airways,2019-04-27,Banglore,Delhi,08:20:00,11:20:00,180,0.0,No Info,7229
10681,Vistara,2019-03-01,Banglore,New Delhi,11:30:00,14:10:00,160,0.0,No Info,12648


In [148]:
flights_cleaned.head()

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-03-24,Banglore,New Delhi,22:20:00,01:10:00,170,0.0,No Info,3897
1,Air India,2019-05-01,Kolkata,Banglore,05:50:00,13:15:00,445,2.0,No Info,7662
2,Jet Airways,2019-06-09,Delhi,Cochin,09:25:00,04:25:00,1140,2.0,No Info,13882
3,Indigo,2019-05-12,Kolkata,Banglore,18:05:00,23:30:00,325,1.0,No Info,6218
4,Indigo,2019-03-01,Banglore,New Delhi,16:50:00,21:35:00,285,1.0,No Info,13302


In [50]:
flights.Airline.unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

- Some of entries has inconsistent values


`Airlines`


In [43]:
(
    flights.Airline.str.replace("Premium economy", "")
    .str.replace("Business", "")
    .str.title()
    .unique()
)

array(['Indigo', 'Air India', 'Jet Airways', 'Spicejet',
       'Multiple Carriers', 'Goair', 'Vistara', 'Air Asia', 'Vistara ',
       'Jet Airways ', 'Multiple Carriers ', 'Trujet'], dtype=object)

`Date_of_Journey`


In [53]:
flights.Date_of_Journey

0        24/03/2019
1         1/05/2019
2         9/06/2019
3        12/05/2019
4        01/03/2019
            ...    
10678     9/04/2019
10679    27/04/2019
10680    27/04/2019
10681    01/03/2019
10682     9/05/2019
Name: Date_of_Journey, Length: 10683, dtype: object

In [55]:
pd.to_datetime(flights.Date_of_Journey, dayfirst=True)

0       2019-03-24
1       2019-05-01
2       2019-06-09
3       2019-05-12
4       2019-03-01
           ...    
10678   2019-04-09
10679   2019-04-27
10680   2019-04-27
10681   2019-03-01
10682   2019-05-09
Name: Date_of_Journey, Length: 10683, dtype: datetime64[ns]

`Source`


In [64]:
flights.Source.unique()

array(['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai'], dtype=object)

`Destination`


In [65]:
flights.Destination.unique()

array(['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Delhi', 'Hyderabad'],
      dtype=object)

`Route`


In [66]:
flights.Route

0                    BLR → DEL
1        CCU → IXR → BBI → BLR
2        DEL → LKO → BOM → COK
3              CCU → NAG → BLR
4              BLR → NAG → DEL
                 ...          
10678                CCU → BLR
10679                CCU → BLR
10680                BLR → DEL
10681                BLR → DEL
10682    DEL → GOI → BOM → COK
Name: Route, Length: 10683, dtype: object

`Departure`


In [71]:
flights.Dep_Time

0        22:20
1        05:50
2        09:25
3        18:05
4        16:50
         ...  
10678    19:55
10679    20:45
10680    08:20
10681    11:30
10682    10:55
Name: Dep_Time, Length: 10683, dtype: object

In [73]:
(flights.Dep_Time.loc[lambda ser: ser.str.contains("[^0-9:]")])

Series([], Name: Dep_Time, dtype: object)

In [75]:
pd.to_datetime(flights.Dep_Time).dt.time

0        22:20:00
1        05:50:00
2        09:25:00
3        18:05:00
4        16:50:00
           ...   
10678    19:55:00
10679    20:45:00
10680    08:20:00
10681    11:30:00
10682    10:55:00
Name: Dep_Time, Length: 10683, dtype: object

`Arrival_Time`


In [78]:
flights.Arrival_Time

0        01:10 22 Mar
1               13:15
2        04:25 10 Jun
3               23:30
4               21:35
             ...     
10678           22:25
10679           23:20
10680           11:20
10681           14:10
10682           19:15
Name: Arrival_Time, Length: 10683, dtype: object

In [83]:
(
    flights.Arrival_Time.loc[lambda ser: ser.str.contains("[^0-9:]")]
    .str.split(" ", n=1)
    .str.get(-1)
    .unique()
)

array(['22 Mar', '10 Jun', '13 Mar', '02 Mar', '10 May', '04 Mar',
       '13 Jun', '28 May', '19 Mar', '07 May', '02 Jun', '16 Jun',
       '19 May', '16 May', '28 Jun', '02 May', '28 Mar', '19 Jun',
       '04 Apr', '25 Mar', '07 Mar', '25 Jun', '07 Jun', '25 May',
       '13 May', '16 Mar', '22 May', '10 Apr', '04 Jun', '20 May',
       '28 Apr', '25 Apr', '10 Mar', '19 Apr', '13 Apr', '02 Apr',
       '23 Mar', '22 Apr', '11 May', '07 Apr', '03 May', '08 Mar',
       '03 Mar', '05 Mar', '22 Jun', '04 May', '26 May', '16 Apr',
       '26 Jun', '29 May', '29 Jun', '29 Mar', '23 May', '17 Jun'],
      dtype=object)

`duration`


In [88]:
flights.Duration

0        2h 50m
1        7h 25m
2           19h
3        5h 25m
4        4h 45m
          ...  
10678    2h 30m
10679    2h 35m
10680        3h
10681    2h 40m
10682    8h 20m
Name: Duration, Length: 10683, dtype: object

In [113]:
(
    flights.Duration.drop(index=6474)
    .str.split(" ", expand=True)
    .set_axis(["hour", "minute"], axis=1)
    .assign(
        hour=lambda df_: (df_.hour.str.replace("h", "").astype(int).mul(60)),
        minute=lambda df_: (df_.minute.str.replace("m", "").fillna(0).astype(int)),
    )
    .sum(axis=1)
)

0         170
1         445
2        1140
3         325
4         285
         ... 
10678     150
10679     155
10680     180
10681     160
10682     500
Length: 10682, dtype: int64

In [115]:
# this is just to validate if conversion happens correctly or not
(
    flights.Duration.drop(index=6474)
    .str.split(" ", expand=True)
    .set_axis(["hour", "minute"], axis=1)
    .assign(
        hour=lambda df_: (df_.hour.str.replace("h", "").astype(int).mul(60)),
        minute=lambda df_: (df_.minute.str.replace("m", "").fillna(0).astype(int)),
    )
    .sum(axis=1)
    .rename("duration_minutes")
    .to_frame()
    .join(flights.Duration.drop(index=6474))
)

,duration_minutes,Duration
0,170,2h 50m
1,445,7h 25m
2,1140,19h
3,325,5h 25m
4,285,4h 45m
...,...,...
10678,150,2h 30m
10679,155,2h 35m
10680,180,3h
10681,160,2h 40m


In [91]:
(flights.Duration.loc[lambda ser: ~ser.str.contains("h")])

6474    5m
Name: Duration, dtype: object

In [94]:
flights.iloc[[6474]]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6474,Air India,6/03/2019,Mumbai,Hyderabad,BOM → GOI → PNQ → HYD,16:50,16:55,5m,2 stops,No info,17327


- The obs index `6474` has duration of 5 mins which is wrong, we will delete this row


`Total_Stops`


In [120]:
flights.Total_Stops.unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [130]:
(
    flights.Total_Stops.replace("non-stop", "0")
    .str.replace(" stops?", "", regex=True)  # " stop" or " stops"
    .pipe(lambda ser: pd.to_numeric(ser))
    # .astype(float)
)

0        0.0
1        2.0
2        2.0
3        1.0
4        1.0
        ... 
10678    0.0
10679    0.0
10680    0.0
10681    0.0
10682    2.0
Name: Total_Stops, Length: 10683, dtype: float64

`additional_info`


In [142]:
flights.Additional_Info.unique()

array(['No info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover', 'No Info',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)

## Spliting the data


In [156]:
flight_final = flights_cleaned.sample(1000)

In [157]:
X = flight_final.drop(columns="price")
y = flight_final.price.copy()

In [158]:
X_, X_test, y_, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(
    X_, y_, test_size=0.2, random_state=42
)

In [159]:
print("x train shape:", X_train.shape, "\ny train shape:", y_train.shape, "\n ")
print("x val shape:", X_val.shape, "\ny val shape:", y_val.shape, "\n")
print("X test shape:", X_test.shape, "\ny test shape:", y_test.shape, "\n ")

x train shape: (640, 9) 
y train shape: (640,) 
 
x val shape: (160, 9) 
y val shape: (160,) 

X test shape: (200, 9) 
y test shape: (200,) 
 


### Export the data


In [160]:
def export_data(X, y, name):
    file_name = f"{name}.csv"
    file_path = os.path.join(PROJECT_DIR, DATA_DIR, file_name)
    X.join(y).to_csv(file_path, index=False)
    return pd.read_csv(file_path).sample(5)

In [161]:
export_data(X_train,y_train,"train_data")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
220,Indigo,2019-04-15,Kolkata,Banglore,19:35:00,23:15:00,220,1.0,No Info,4226
518,Air India,2019-06-24,Delhi,Cochin,10:55:00,07:40:00,1245,2.0,No Info,11753
487,Indigo,2019-06-27,Kolkata,Banglore,04:40:00,07:15:00,155,0.0,No Info,4804
429,Air India,2019-06-24,Delhi,Cochin,10:00:00,19:15:00,555,1.0,No Info,9194
24,Jet Airways,2019-03-18,Banglore,New Delhi,16:55:00,23:00:00,365,1.0,In-flight meal not included,11087


In [162]:
export_data(X_test, y_test, "test_data")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
108,Multiple Carriers,2019-06-09,Delhi,Cochin,08:00:00,21:00:00,780,1.0,No Info,7408
69,Jet Airways,2019-06-12,Kolkata,Banglore,16:30:00,10:05:00,1055,1.0,In-flight meal not included,8529
123,Indigo,2019-03-21,Delhi,Cochin,07:35:00,22:20:00,885,1.0,No Info,6558
56,Vistara,2019-06-01,Chennai,Kolkata,17:45:00,20:05:00,140,0.0,No Info,11982
160,Indigo,2019-03-21,Banglore,New Delhi,04:00:00,06:50:00,170,0.0,No Info,4777


In [163]:
export_data(X_val, y_val, "val_data")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
147,Multiple Carriers,2019-06-27,Delhi,Cochin,10:00:00,19:00:00,540,1.0,No Info,12192
42,Indigo,2019-06-24,Banglore,Delhi,18:55:00,21:50:00,175,0.0,No Info,4823
89,Spicejet,2019-05-12,Banglore,Delhi,09:30:00,12:20:00,170,0.0,No check-in baggage included,4319
2,Air India,2019-05-06,Kolkata,Banglore,16:45:00,12:00:00,1155,2.0,No Info,14120
28,Multiple Carriers,2019-06-12,Delhi,Cochin,09:00:00,19:15:00,615,1.0,No Info,14848
